# Chapter 4 GLMのモデル選択 ---AICのモデルの予測と良さ---

In [1]:
date()

[1] "Sun Oct 16 21:27:31 2016"

## 4.1 データはひとつ，モデルはたくさん

* 一定モデル（$k = 1$）
* `f`モデル（$k = 2$）
* `x`モデル（$k = 2$）
* `x + f` モデル（$k = 3$）

## 4.2 統計モデルの当てはまりの悪さ: 逸脱度

* 対数尤度: $\log L$
* 最大対数尤度: $\log L^\ast$
* 逸脱度 $D$

$$ D = -2 \log L^\ast $$

In [2]:
sapply(c("pipeR", "dplyr", "tidyr", "ggplot2", "readr"), require, character.only = TRUE)

Loading required package: pipeR
Warning message:
“package ‘pipeR’ was built under R version 3.2.4”Loading required package: dplyr

Attaching package: ‘dplyr’

 以下のオブジェクトは ‘package:stats’ からマスクされています: 

     filter, lag 

 以下のオブジェクトは ‘package:base’ からマスクされています: 

     intersect, setdiff, setequal, union 

Loading required package: tidyr
Loading required package: ggplot2
Loading required package: readr


pipeR   dplyr   tidyr ggplot2   readr 
   TRUE    TRUE    TRUE    TRUE    TRUE

In [3]:
d <- read_csv("data/chap03/data3a.csv")
str(d)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	100 obs. of  3 variables:
 $ y: int  6 6 6 12 10 4 9 9 9 11 ...
 $ x: num  8.31 9.44 9.5 9.07 10.16 ...
 $ f: chr  "C" "C" "C" "C" ...


一定モデル

In [8]:
d %>>% mutate(f = factor(f)) -> d

In [9]:
glm(y ~ 1, data = d, family = "poisson")


Call:  glm(formula = y ~ 1, family = "poisson", data = d)

Coefficients:
(Intercept)  
      2.058  

Degrees of Freedom: 99 Total (i.e. Null);  99 Residual
Null Deviance:	    89.51 
Residual Deviance: 89.51 	AIC: 477.3

fモデル

In [10]:
glm(y ~ f, data = d, family = "poisson")


Call:  glm(formula = y ~ f, family = "poisson", data = d)

Coefficients:
(Intercept)           fT  
    2.05156      0.01277  

Degrees of Freedom: 99 Total (i.e. Null);  98 Residual
Null Deviance:	    89.51 
Residual Deviance: 89.48 	AIC: 479.3

xモデル

In [11]:
glm(y ~ x, data = d, family = "poisson")


Call:  glm(formula = y ~ x, family = "poisson", data = d)

Coefficients:
(Intercept)            x  
    1.29172      0.07566  

Degrees of Freedom: 99 Total (i.e. Null);  98 Residual
Null Deviance:	    89.51 
Residual Deviance: 84.99 	AIC: 474.8

x + fモデル

In [12]:
glm(y ~ x + f, data = d, family = "poisson")


Call:  glm(formula = y ~ x + f, family = "poisson", data = d)

Coefficients:
(Intercept)            x           fT  
    1.26311      0.08007     -0.03200  

Degrees of Freedom: 99 Total (i.e. Null);  97 Residual
Null Deviance:	    89.51 
Residual Deviance: 84.81 	AIC: 476.6

Residual Deviance は $D$ - 最小逸脱度（full model）

full model の最大対数尤度$\log L ^\ast$は，

In [13]:
dpois(d$y, lambda = d$y) %>>% log %>>% sum

[1] -192.8898

なので，逸脱度$D$は，

In [14]:
dpois(d$y, lambda = d$y) %>>% log %>>% sum %>>% {(.) * (-2)} -> D.full
D.full

[1] 385.7795

$D$ - D.full = Resid. Dev. 

In [16]:
glm(y ~ 1, data = d, family = "poisson") -> fit.c
logLik(fit.c)

'log Lik.' -237.6432 (df=1)

In [17]:
-2 * -237.6432 - D.full

[1] 89.50689

In [19]:
fit.c$deviance

[1] 89.50694

## 4.3 モデル選択基準 AIC

* 予測の良さを重視する
* AIC最小が良いモデル

$$ 
\begin{eqnarray} 
    \mathrm{AIC} &=& -2 \{ \text{（最大対数尤度）} - \text{（最尤推定したパラメータ数）}\} \\
     &=& -2 (\log L^\ast - k) \\
     &=& D + 2 k
\end{eqnarray}
$$


## 4.4 AICを説明するためのまた別の例題

## 4.5 なぜAICでモデル選択してよいのか

### 4.5.1 統計モデルの予測の良さ: 平均対数尤度

### 4.5.2 最大対数尤度のバイアス補正

### 4.5.3 ネストしているGLM間のAIC比較

## 4.6 まとめと参考文献

In [21]:
ls()

[1] "d"      "D.full" "fit.c"

In [23]:
load("data/chap04/data.RData", dum <- new.env())
ls(dum)

[1] "m.data"     "m.mle"      "v.ll"       "v.mean.mle" "v.mle"

In [33]:
dum$m.data %>>% str()

 num [1:50, 1:200] 7 4 7 8 3 7 5 5 4 6 ...


In [38]:
dum$m.data[,1]

[1]  7  4  7  8  3  7  5  5  4  6  6  9  6  5 10  9 11  5 14  8 11 10  6  5  3
[26]  6  4  7  9  8  7  7  8  2  4  8  8  4  7 12  5  9  8  5  7  2  5  4 10  6

In [39]:
devtools::session_info()

Session info -------------------------------------------------------------------
Packages -----------------------------------------------------------------------


 setting  value                       
 version  R version 3.2.3 (2015-12-10)
 system   x86_64, darwin13.4.0        
 ui       X11                         
 language (EN)                        
 collate  ja_JP.UTF-8                 
 tz       Asia/Tokyo                  
 date     2016-10-16                  

 package    * version    date       source                            
 assertthat   0.1        2013-12-06 CRAN (R 3.2.0)                    
 colorspace   1.2-6      2015-03-11 CRAN (R 3.2.0)                    
 crayon       1.3.2      2016-06-28 CRAN (R 3.2.5)                    
 DBI          0.4-1      2016-05-08 CRAN (R 3.2.5)                    
 devtools     1.12.0     2016-06-24 CRAN (R 3.2.5)                    
 digest       0.6.10     2016-08-02 CRAN (R 3.2.5)                    
 dplyr      * 0.4.3      2015-09-01 CRAN (R 3.2.0)                    
 evaluate     0.9        2016-04-29 CRAN (R 3.2.5)                    
 ggplot2    * 2.1.0.9001 2016-10-01 Github (hadl